In [1]:
import pickle
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import copy

In [2]:
teams = ['BKN', 'MIL', 'GSW', 'LAL', 'IND', 'CHA', 'CHI', 'DET',
          'WAS', 'TOR', 'BOS', 'NYK', 'CLE', 'MEM', 'PHI', 'NOP',
          'HOU','MIN', 'ORL', 'SAS', 'OKC', 'UTA', 'SAC', 'POR',
          'DEN', 'PHX', 'DAL', 'ATL', 'MIA', 'LAC']
window = 4 # 4 samples per game (one sample / quarter)

In [3]:
file_name = '../team-profiles/profiles.pkl'
with open(file_name, 'rb') as file:
    # Use pickle.load() to read the list from the file
    team_profiles = pickle.load(file)

In [4]:
def print_params(params):
    M_, N_, E_, R_ = params['M'], params['N'], params['E'], params['R']

    print(f'M = {np.round(M_, 2)}')
    print(f'N = {np.round(N_, 2)}')
    print(f'E = {np.round(E_, 2)}')
    for p in players:
        print(f'R[{p}] = {np.round(R_[p], 2)}')

In [5]:
N_players = 5
n_components = 3 # num of hidden states
n_features = 3 # num of observed states
O_symbols = [0, 1, 2] # under-, avg-, over- performance
H_symbols = [0, 1, 2] # corresponding mental states
initial_dist = np.array([0, 1, 0])

In [6]:
def cond(player, h1, h2, t, M_, N_, R_): # P(H_t^player = h1 | H_{t-1}^player = h2, O_{t-1})
    # Requires M_, N_, R_, Os
    v = [N_[h2][h1]]
    for teammate in players:
        v.append(M_[Os[teammate][t-1]][h1])
            
    v = np.array(v)
    return np.dot(R_[player], v)  

In [7]:
def calculate_forward(M_, N_, E_, R_):
    alpha = {p: [] for p in players}
    alpha_help = {p: [] for p in players}
    
    # Initialize forward parameters
    for p in players:
        alpha_help[p].append(initial_dist)
    for p in players:
        arr = np.array([E_[h][Os[p][0]] * alpha_help[p][0][h] for h in H_symbols])
        arr /= np.sum(arr)
        alpha[p].append(arr)

    # Compute forward parameters (bottom-up)
    for p in players:
        for t in range(1, T):
            if t % 4 == 0:
                alpha[p].append(initial_dist)
                alpha_help[p].append(initial_dist)
                continue
                
            arr = [sum([cond(p, h, h_, t, M_, N_, R_) * alpha[p][t-1][h_] for h_ in H_symbols]) for h in H_symbols]
            arr = np.array(arr)
            alpha_help[p].append(arr)
            
            arr = [E_[h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols]
            arr = np.array(arr)
            arr /= np.sum(arr)
            alpha[p].append(arr)
            
    return alpha, alpha_help

In [8]:
def rename_keys(original_dict, key_mapping):
    new_dict = {}
    for old_key, new_key in key_mapping.items():
        if old_key in original_dict:
            new_dict[new_key] = original_dict[old_key]
    return new_dict

In [9]:
def likelihood(params): # Calculate log P(O) (given the model parameters)
    M, N, E, R = params['M'], params['N'], params['E'], params['R']
    
    # Caluclate forward parameters
    alpha, alpha_help = calculate_forward(M, N, E, R)
    obj = 0
    for p in players:
        for t in range(T):
            if isinstance(E, dict):
                obj += -np.log(sum([E[p][h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols])) # Pr(O_t^i | O_{1:t-1})
            else:
                obj += -np.log(sum([E[h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols])) # Pr(O_t^i | O_{1:t-1})

    return obj

def likelihood_test(params_dict):
    total_params = len(params_dict)
    # Given a list of model parameters returns the model parameters that achieve the highest likelihood
    val_opt = 1e+100
    i = 1
    for key in params_dict:
        params = params_dict[key]
        params['R'] = rename_keys(params['R'], key_mapping)
        val = likelihood(params)
        if i % 1000 == 0:
            print(f'{i} / {total_params}, val = {np.round(val, 2)}')
        i += 1
        if val < val_opt:
            val_opt = val
            params_opt = params
    return params_opt

In [10]:
Os_teams = {team: dict() for team in teams}
for team in teams:
    with open(f'../team-data/observations/{team}_observations.pickle', 'rb') as file:
            observations = pickle.load(file)
    
    k = list(observations.keys())[0]
    players = list(observations[k].keys())
    
    Os_teams[team] = {p: [] for p in players}
    for game_id in observations:
        game_stats = observations[game_id]
        for p in players:
            Os_teams[team][p].extend(game_stats[p])

In [11]:
for team in teams:
    print(f'Team = {team}')
    try:
        players = list(Os_teams[team].keys())[:N_players]
        i = 1
        key_mapping = dict()
        for p in players:
            key_mapping[f'player{i}'] = players[i-1]
            i += 1

        Os = Os_teams[team]
        T = len(Os[players[0]])
        profiles = copy.deepcopy(team_profiles)
        team_params = likelihood_test(profiles)

        # Open the file in binary write mode
        with open(f'./profiles/{team}_profile.pkl', 'wb') as file:
            pickle.dump(team_params, file)
           
    except:
        print(f'Error for team {team}')

# print(f'=== Params for {team} ===')
# print_params(team_params)

Team = BKN
1000 / 5667, val = 1109.25
2000 / 5667, val = 1068.29
3000 / 5667, val = 1133.18
4000 / 5667, val = 1157.75
5000 / 5667, val = 1139.1
Team = MIL
1000 / 5667, val = 2072.51
2000 / 5667, val = 2132.31
3000 / 5667, val = 2130.51
4000 / 5667, val = 2160.37
5000 / 5667, val = 2079.25
Team = GSW
1000 / 5667, val = 2460.38
2000 / 5667, val = 2482.14
3000 / 5667, val = 2480.29
4000 / 5667, val = 2551.32
5000 / 5667, val = 2449.14
Team = LAL
1000 / 5667, val = 1800.41
2000 / 5667, val = 1832.32
3000 / 5667, val = 1832.38
4000 / 5667, val = 1883.83
5000 / 5667, val = 1810.99
Team = IND
1000 / 5667, val = 1527.58
2000 / 5667, val = 1552.04
3000 / 5667, val = 1547.01
4000 / 5667, val = 1554.58
5000 / 5667, val = 1531.94
Team = CHA
1000 / 5667, val = 1929.59
2000 / 5667, val = 1957.72
3000 / 5667, val = 1953.1
4000 / 5667, val = 2013.55
5000 / 5667, val = 1932.54
Team = CHI
1000 / 5667, val = 1965.66
2000 / 5667, val = 2020.32
3000 / 5667, val = 2019.29
4000 / 5667, val = 2005.12
5000 / 